### 4.10.1 Démarrage de la session Spark

In [1]:
# L'exécution de cette cellule démarre l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1713982757102_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Affichage des informations sur la session en cours et liens vers Spark UI</u> :

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1713982757102_0004,pyspark,idle,Link,Link,None,✔


### 4.10.2 Installation des packages

Les packages nécessaires ont été installé via l'étape de **bootstrap** à l'instanciation du serveur.
```bash
#!/bin/bash
# install requirements on all cluster nodes, not only master
# update pip to prevent pyarrow installation error
sudo python3 -m pip install -U setuptools
sudo python3 -m pip install -U pip
sudo python3 -m pip install wheel
sudo python3 -m pip install pillow
# same version of tensorflow and keras to avoid module not found error
sudo python3 -m pip install tensorflow==2.11.0
sudo python3 -m pip install keras==2.11.0
sudo python3 -m pip install pandas==1.2.5
sudo python3 -m pip install pyarrow
sudo python3 -m pip install boto3
sudo python3 -m pip install s3fs
sudo python3 -m pip install fsspec

```

### 4.10.3 Import des librairies

In [3]:
import io

import numpy as np
import pandas as pd
from PIL import Image
from pyspark.ml.feature import PCA, StandardScaler, VectorAssembler
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import PandasUDFType, col, element_at, pandas_udf, split, udf
from pyspark.sql.types import *
from tensorflow.keras import Model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4.10.4 Définition des PATH pour charger les images et enregistrer les résultats

Nous accédons directement à nos **données sur S3** comme si elles étaient **stockées localement**.

In [4]:
# define s3 path for reading data and writing results
BUCKET = "s3://carl-p8-v2"
PATH_Data = BUCKET + "/sample_test"
PATH_Result = BUCKET + "/Results"

print(
    "PATH:        "
    + BUCKET
    + "\nPATH_Data:   "
    + PATH_Data
    + "\nPATH_Result: "
    + PATH_Result
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://carl-p8-v2
PATH_Data:   s3://carl-p8-v2/sample_test
PATH_Result: s3://carl-p8-v2/Results

### 4.10.5 Traitement des données

#### 4.10.5.1 Chargement des données

In [5]:
# Load images using Spark
images = (
    spark.read.format("binaryFile")
    .option("pathGlobFilter", "*.jpg")
    .option("recursiveFileLookup", "true")
    .load(PATH_Data)
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
print((images.count(), len(images.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(2142, 4)

In [7]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://carl-p8-v2/s...|2024-04-21 17:02:55|  5883|[FF D8 FF E0 00 1...|
|s3://carl-p8-v2/s...|2024-04-21 17:02:55|  5851|[FF D8 FF E0 00 1...|
|s3://carl-p8-v2/s...|2024-04-21 17:02:54|  5815|[FF D8 FF E0 00 1...|
|s3://carl-p8-v2/s...|2024-04-21 17:02:54|  5810|[FF D8 FF E0 00 1...|
|s3://carl-p8-v2/s...|2024-04-21 17:02:54|  5801|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

<u>Je ne conserve que le **path** de l'image et j'ajoute <br />
    une colonne contenant les **labels** de chaque image</u> :

In [8]:
# keep only path and add label column
images = images.withColumn("label", element_at(split(images["path"], "/"), -2))
print(images.printSchema())
print(images.select("path", "label").show(5, False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+---------------------------------------------------+---------+
|path                                               |label    |
+---------------------------------------------------+---------+
|s3://carl-p8-v2/sample_test/Nectarine/r_33_100.jpg |Nectarine|
|s3://carl-p8-v2/sample_test/Nectarine/r_32_100.jpg |Nectarine|
|s3://carl-p8-v2/sample_test/Nectarine/r_326_100.jpg|Nectarine|
|s3://carl-p8-v2/sample_test/Nectarine/r_322_100.jpg|Nectarine|
|s3://carl-p8-v2/sample_test/Nectarine/r_324_100.jpg|Nectarine|
+---------------------------------------------------+---------+
only showing top 5 rows

None

In [9]:
images.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+---------+
|                path|   modificationTime|length|             content|    label|
+--------------------+-------------------+------+--------------------+---------+
|s3://carl-p8-v2/s...|2024-04-21 17:02:55|  5883|[FF D8 FF E0 00 1...|Nectarine|
|s3://carl-p8-v2/s...|2024-04-21 17:02:55|  5851|[FF D8 FF E0 00 1...|Nectarine|
|s3://carl-p8-v2/s...|2024-04-21 17:02:54|  5815|[FF D8 FF E0 00 1...|Nectarine|
|s3://carl-p8-v2/s...|2024-04-21 17:02:54|  5810|[FF D8 FF E0 00 1...|Nectarine|
|s3://carl-p8-v2/s...|2024-04-21 17:02:54|  5801|[FF D8 FF E0 00 1...|Nectarine|
|s3://carl-p8-v2/s...|2024-04-21 17:02:55|  5801|[FF D8 FF E0 00 1...|Nectarine|
|s3://carl-p8-v2/s...|2024-04-21 17:02:54|  5798|[FF D8 FF E0 00 1...|Nectarine|
|s3://carl-p8-v2/s...|2024-04-21 17:02:54|  5795|[FF D8 FF E0 00 1...|Nectarine|
|s3://carl-p8-v2/s...|2024-04-21 17:02:55|  5793|[FF D8 FF E0 00 1...|Nectarine|
|s3://carl-p8-v2/s...|2024-0

#### 4.10.5.2 Préparation du modèle

In [10]:
# use MobileNetV2 to extract features
model = MobileNetV2(weights="imagenet", include_top=True, input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# remove top layer
new_model = Model(inputs=model.input, outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Broadcast the model's weights to the workers to ensure that they are always available.
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [14]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights="imagenet", include_top=True, input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input, outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 4.10.5.3 Définition du processus de chargement des images <br/> et application de leur featurisation à travers l'utilisation de pandas UDF

In [15]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)


def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)


@pandas_udf("array<float>", PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    """
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    """
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt/yarn/usercache/livy/appcache/application_1713982757102_0004/container_1713982757102_0004_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

#### 4.10.5.4 Exécutions des actions d'extractions de features

In [16]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
images.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+---------+
|                path|   modificationTime|length|             content|    label|
+--------------------+-------------------+------+--------------------+---------+
|s3://carl-p8-v2/s...|2024-04-21 17:02:55|  5883|[FF D8 FF E0 00 1...|Nectarine|
|s3://carl-p8-v2/s...|2024-04-21 17:02:55|  5851|[FF D8 FF E0 00 1...|Nectarine|
|s3://carl-p8-v2/s...|2024-04-21 17:02:54|  5815|[FF D8 FF E0 00 1...|Nectarine|
|s3://carl-p8-v2/s...|2024-04-21 17:02:54|  5810|[FF D8 FF E0 00 1...|Nectarine|
|s3://carl-p8-v2/s...|2024-04-21 17:02:54|  5801|[FF D8 FF E0 00 1...|Nectarine|
|s3://carl-p8-v2/s...|2024-04-21 17:02:55|  5801|[FF D8 FF E0 00 1...|Nectarine|
|s3://carl-p8-v2/s...|2024-04-21 17:02:54|  5798|[FF D8 FF E0 00 1...|Nectarine|
|s3://carl-p8-v2/s...|2024-04-21 17:02:54|  5795|[FF D8 FF E0 00 1...|Nectarine|
|s3://carl-p8-v2/s...|2024-04-21 17:02:55|  5793|[FF D8 FF E0 00 1...|Nectarine|
|s3://carl-p8-v2/s...|2024-0

In [18]:
# partition the images DataFrame to 4 partitions
# apply the UDF to the images DataFrame
features_df = images.repartition(4).select(
    col("path"), col("label"), featurize_udf("content").alias("features")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
type(features_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>

In [20]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://carl-p8-v2/Results

In [21]:
# save the result to parquet
features_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
features_df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(path='s3://carl-p8-v2/sample_test/Nectarine/61_100.jpg', label='Nectarine', features=[0.5543004870414734, 0.0, 0.017258351668715477, 0.0, 0.0, 0.0, 0.7030156850814819, 0.23641814291477203, 0.04427652060985565, 0.0, 0.0, 0.0, 0.0, 0.013368879444897175, 0.0, 0.0003095451684202999, 0.0, 0.0, 0.05343972519040108, 0.024294288828969002, 0.0, 0.12039019167423248, 0.04006500542163849, 0.0, 0.0388205386698246, 0.0, 0.0, 0.0, 0.0, 0.21158453822135925, 0.09676539897918701, 0.0, 0.0, 0.0, 0.0, 0.8954373002052307, 2.465944528579712, 1.6249622106552124, 0.5903096199035645, 0.0, 0.0, 0.0, 0.7369627952575684, 0.0, 0.0, 0.0, 1.0006237030029297, 0.01723293401300907, 0.1619802862405777, 0.05702539533376694, 0.01628267578780651, 1.0910766124725342, 0.0, 0.0, 0.0, 2.494385004043579, 0.0, 0.0, 0.0, 0.3838648498058319, 0.0, 0.30083030462265015, 0.0, 0.0, 0.2299467772245407, 1.3707249164581299, 0.07132822275161743, 0.10845359414815903, 0.0, 1.9026635885238647, 0.0, 0.054265741258859634, 0.0, 1.35493350028

### 4.10.6 Chargement des données enregistrées et validation du résultat

In [23]:
# read the result back
df = pd.read_parquet(PATH_Result, engine="pyarrow")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                path  ...                                           features
0  s3://carl-p8-v2/sample_test/Cantaloupe 1/r_124...  ...  [0.013953569, 0.0, 0.0, 0.0, 0.0, 0.5506894, 0...
1   s3://carl-p8-v2/sample_test/Strawberry/3_100.jpg  ...  [1.5514314, 0.0, 0.0, 0.0, 0.03954255, 0.14615...
2  s3://carl-p8-v2/sample_test/Cantaloupe 1/r_137...  ...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.22503383, 0.332275...
3  s3://carl-p8-v2/sample_test/Nectarine/r_9_100.jpg  ...  [0.34195328, 0.0, 0.0, 0.0, 0.0, 0.0, 0.983408...
4  s3://carl-p8-v2/sample_test/Nectarine/r_321_10...  ...  [0.34154153, 0.0, 0.0, 0.0, 0.0, 0.0, 1.401063...

[5 rows x 3 columns]

In [25]:
df.loc[0, "features"].shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1280,)

In [26]:
df.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(2142, 3)

# 5. PCA

In [27]:
# "features" column created by featurize_udf is of type ArrayType
# pyspark.mk needs a VectorUDT instead

# Define UDF to convert ArrayType to VectorUDT
array_to_vector_udf = udf(lambda vs: Vectors.dense(vs), VectorUDT())

# Apply the UDF to the "features" column
features_df = features_df.withColumn("features", array_to_vector_udf("features"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# scale the data
scaler = StandardScaler(
    inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=True
)
scalerModel = scaler.fit(features_df)
scaledData = scalerModel.transform(features_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
# apply PCA
N_PC = 100
pca = PCA(k=N_PC, inputCol="scaledFeatures", outputCol="pcaFeatures")
pcaModel = pca.fit(scaledData)
pcaData = pcaModel.transform(scaledData).select("pcaFeatures")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
# print((pcaData.count(), len(pcaData.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
expVariance = pcaModel.explainedVariance
# print(expVariance)
# pd.Series(v for v in expVariance).cumsum()

# total explained Variance
print(
    f"{N_PC} principal components explain {pd.Series(v for v in expVariance).sum()} % of the variance"
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

100 principal components explain 0.893703977907596 % of the variance

In [32]:
features_df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(path='s3://carl-p8-v2/sample_test/Cantaloupe 1/r_74_100.jpg', label='Cantaloupe 1', features=DenseVector([0.0286, 0.0, 0.0, 0.0, 0.0, 0.1058, 0.0091, 0.0324, 0.2257, 0.0, 0.0, 0.1959, 0.5782, 0.0, 0.0385, 1.876, 0.0, 0.0143, 0.036, 0.0, 0.0, 0.0, 0.0437, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4089, 0.2881, 0.8129, 0.0, 0.0, 0.0, 0.0, 0.0, 1.1513, 1.8927, 3.9957, 0.0, 0.0006, 0.0, 0.0, 0.0575, 0.0994, 0.0, 1.0817, 2.0527, 0.0629, 0.0164, 0.0265, 0.0116, 0.0, 0.0, 0.0, 1.409, 0.2121, 0.0, 0.0, 0.0, 0.0, 0.3829, 0.0945, 0.0, 0.36, 0.0873, 0.246, 0.0, 0.0755, 1.4731, 0.0, 0.0, 0.0, 0.0062, 0.361, 0.0, 0.0241, 0.2382, 0.0, 0.0, 0.0, 0.3742, 0.0, 0.0162, 0.0253, 0.1141, 0.0, 0.0, 0.0279, 1.2891, 0.211, 0.2493, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0859, 0.0334, 0.0245, 0.1333, 0.7358, 0.0, 0.0, 0.0, 0.0141, 0.5824, 0.9385, 0.0, 0.0, 0.0, 0.0047, 0.0, 1.8022, 0.2103, 0.1035, 0.1215, 0.0132, 0.0267, 0.048, 0.089, 0.0163, 0.0, 0.8581, 0.2124, 0.027, 2.4896, 1.1592, 0.5917, 0.2381, 0.0, 1.9542, 0.0, 0.0, 0.0, 0.03

In [33]:
pcaData.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(pcaFeatures=DenseVector([4.817, 10.1565, -8.7916, -2.3085, 0.6933, -5.8004, 2.7017, -4.1868, -3.7349, 5.9635, -0.1706, 6.6071, 0.1985, 4.8581, 1.3778, -13.1918, -5.7252, 0.2149, -6.0013, -4.6075, -3.2325, -6.0244, 0.1507, -2.8739, 0.6538, 3.2298, -0.7357, 0.5593, 3.5934, 1.6261, -0.8908, 1.9552, 0.2362, 0.477, -0.7897, 0.0026, -0.0494, -0.5544, 0.0044, 0.0847, -0.5627, -0.4011, 0.2914, 0.3929, -0.3706, 0.444, 0.4852, -0.9169, 2.3752, -0.0589, 0.381, 0.3918, 0.411, 0.0898, 0.3579, 0.3258, -0.5464, -0.865, -0.7167, 0.8015, -0.4396, 0.5868, 0.8955, 0.1206, -0.2741, -0.4252, 0.0626, -0.5955, -0.1078, -1.4361, -0.3027, 1.0371, 0.9828, -0.2446, 0.6892, 0.1384, -0.7541, 0.2422, -0.2402, -0.8682, 0.0566, -0.4422, 0.3927, -0.2899, 0.3739, 1.6487, -0.1699, -1.0403, -0.5595, 1.0256, 0.1569, 0.0783, 0.4164, 0.5777, -0.0193, -0.3478, -0.8122, -0.8666, -0.3156, 0.3367]))

# 6. Save PCA results

In [34]:
def split_array_to_list(col):
    def to_list(v):
        return v.toArray().tolist()
    return udf(to_list, ArrayType(DoubleType()))(col)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
# split feature vector into new columns
pcaSplit = pcaData.withColumn("pcaFeatures", split_array_to_list(col("pcaFeatures"))).select(
    col("*"), *[col("pcaFeatures")[i] for i in range(N_PC)]
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# concat pca features to image dataframe
results_concat = features_df.join(pcaSplit)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
results_concat.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(path='s3://carl-p8-v2/sample_test/Cantaloupe 1/r_124_100.jpg', label='Cantaloupe 1', features=DenseVector([0.014, 0.0, 0.0, 0.0, 0.0, 0.5507, 0.1115, 0.0, 0.0813, 0.0, 0.0, 0.0282, 0.0, 0.0, 0.0277, 2.242, 0.0, 0.0266, 0.0387, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0074, 0.381, 0.2226, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5949, 0.9085, 3.4851, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0561, 0.0, 0.3697, 1.5789, 0.0987, 0.0, 0.005, 0.7226, 0.0, 0.0, 0.0, 1.1808, 0.1518, 0.0, 0.0, 0.0, 0.0, 0.0699, 0.0006, 0.0, 0.2282, 0.0237, 0.0015, 0.0027, 0.0, 0.7861, 0.0, 0.0, 0.0, 0.0022, 1.2454, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2586, 0.0, 0.0, 0.0, 0.0205, 0.0, 0.0, 0.0, 1.5038, 0.13, 0.0136, 0.0, 0.0, 0.0816, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6053, 0.0026, 0.0, 0.0, 0.0, 0.0, 0.6059, 1.0926, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4723, 0.0, 0.0, 0.3476, 0.0, 0.0085, 0.6557, 0.2722, 0.052, 0.0, 1.3121, 0.0, 0.06, 1.2653, 0.7882, 0.3767, 0.007, 0.0, 2.2866, 0.0, 0.0, 0.8465, 0.0, 0.0, 0.0, 0.666, 0.0046, 0.5948, 0.0, 0.0, 0.0251, 

In [41]:
# save PCA result to csv
results_concat.repartition(4).toPandas().to_csv(PATH_Result + "/pcaFeatures_v2.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o484.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 58.0 failed 4 times, most recent failure: Lost task 2.3 in stage 58.0 (TID 1014) (ip-172-31-44-6.eu-north-1.compute.internal executor 19): ExecutorLostFailure (executor 19 exited caused by one of the running tasks) Reason: Container from a bad node: container_1713982757102_0004_01_000041 on host: ip-172-31-44-6.eu-north-1.compute.internal. Exit status: 137. Diagnostics: [2024-04-24 20:26:15.801]Container killed on request. Exit code is 137
[2024-04-24 20:26:15.802]Container exited with a non-zero exit code 137. 
[2024-04-24 20:26:15.802]Killed by external signal
.
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:3067)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:3003)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abor

In [39]:
pcaData.toPandas().to_csv(PATH_Result + "/pcaFeatures_v1.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…